In [2]:
import re
import pandas as pd
import os
import csv

folder_path = "/content/drive/MyDrive/opendid_test"
files = os.listdir(folder_path)

sorted_files = sorted(files)# 按照文件名的字典順序進行排序

column_labels = ['filename', 'start', 'content']
output = pd.DataFrame(columns = column_labels)
j=0

for file_name in sorted_files:
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    pattern = re.compile(r'(\n)|(?<!Dr)(\. )(?![^\(]*\))(?![^()]*\))(?![A-Z])')
    sentences = re.split(pattern, content) #(分割條件,檔案)
    sentences = [item for item in sentences if item is not None]

    file_name=os.path.splitext(file_name)[0]

    current_position = 0    #當前位置

    for i in range(0, len(sentences)-1, 2):
        sentence = sentences[i] + sentences[i + 1]  # 合併句點和後面的空白或換行符號)

        if ((sentence.isspace() is False) and sentence[0].isspace()):
              num_spaces = len(sentence) - len(sentence.lstrip())
              current_position+=num_spaces
              sentence = sentence[num_spaces:]

        if ((sentence[len(sentence)-2:] == '. ') or (sentence[len(sentence)-1:] == '\n')):

            start_position = current_position  # 起始位置改為目前位置
            end_position = start_position + len(sentence)

            output.loc[j, "filename"] = str(file_name)
            output.loc[j, "start"] = str(start_position)
            output.loc[j, "content"] = str(sentence[:len(sentence)-1])

            current_position = end_position
            if not output.loc[j]["content"].strip():
             output = output.drop(j)
            j+=1

output = output.reset_index(drop=True)
output['content'] = output['content'].str.replace('\t',' ')

output['content'] = output['content'].str.strip()
output.to_csv('/content/split.tsv', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\', doublequote=False)